In [2]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense
import keras
from tensorflow.keras import Sequential
import keras_metrics as km
import tensorflow as tf


# Create Models

## 1.0 Resnet model

In [3]:
num_classes = 2 # specifies the number of classes to be classified

# Then add the new models 

new_model = Sequential()
new_model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))
new_model.add(Dense(num_classes, activation='softmax'))

# since first layer is already trained, restrict/freeze its retraining

new_model.layers[0].trainable = False
new_model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 2)                 4098      
                                                                 
Total params: 23591810 (90.00 MB)
Trainable params: 4098 (16.01 KB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


## 2.0 VGG16 Model

In [4]:
v16_model = Sequential()
#Add the VGG16 model to our new sequential
v16_model.add(VGG16(include_top=False, pooling='avg', weights='imagenet'))
#add a dense layer to this new model after the convolutions.
v16_model.add(Dense(num_classes, activation='softmax'))
#freeze the top layers of our model (vgg16 layers)
v16_model.layers[0].trainable = False
v16_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense_1 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 14715714 (56.14 MB)
Trainable params: 1026 (4.01 KB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


## 3.0 Mobile net

In [5]:
mobile_net = Sequential()
mobile_net.add(MobileNet(include_top=False, pooling='avg', weights='imagenet'))
#add a dense layer to this new model after the convolutions.
mobile_net.add(Dense(num_classes, activation='softmax'))
#freeze the top layers of our model (vgg16 layers)
mobile_net.layers[0].trainable = False
mobile_net.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functi  (None, 1024)              3228864   
 onal)                                                           
                                                                 
 dense_2 (Dense)             (None, 2)                 2050      
                                                                 
Total params: 3230914 (12.32 MB)
Trainable params: 2050 (8.01 KB)
Non-trainable params: 3228864 (12.32 MB)
_________________________________________________________________


# Compile models

In [6]:
# Compile resnet model model
precision = km.categorical_precision()
new_model.compile (optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
v16_model.compile (optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
mobile_net.compile (optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])


# Load Data and perfom Augmentation using data generator

In [7]:
#load the image data

#first, declare the required image size 
image_size = 150
# The image data generator is used to augment our dataset to make sure that our model does not see the same image twice. This is especially important given that we have a relatively small dataset, hence our model is prone to overfitting
#Entropic capacity of a model. How much capacity is your model allowed to store
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input,
                                    rotation_range=40,
                                    zoom_range=0.2,
                                    width_shift_range = 0.2,
                                    height_shift_range=0.2,
                                    horizontal_flip=True,
                                    vertical_flip=False)

train_generator = data_generator.flow_from_directory('../input/Cats_and_dogs/training_set/training_set', target_size=(image_size, image_size),
                                                     batch_size = 200, 
                                                     class_mode = 'categorical')
validation_generator = data_generator.flow_from_directory('../input/Cats_and_dogs/test_set/test_set', target_size=(image_size, image_size),
                                                     batch_size = 100, 
                                                     class_mode = 'categorical')


Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


# Fit and adjust the models

In [8]:
#fit the model to the data 
validation_steps= validation_generator.n // validation_generator.batch_size
print(validation_steps)
print('Fitting the Resnet model')
new_model.fit(train_generator, epochs=2, steps_per_epoch=6, validation_data=validation_generator, validation_steps=validation_steps)
print('Fitting the VGG16 model')
v16_model.fit(train_generator, epochs= 2, steps_per_epoch=6, validation_data=validation_generator, validation_steps=validation_steps)
print('Fitting the Mobile net model')
mobile_net.fit(train_generator,epochs=2, steps_per_epoch=6, validation_data=validation_generator, validation_steps=validation_steps)



20
Fitting the Resnet model
Epoch 1/2


6/6 [==============================] - 118s 18s/step - loss: 1.1969 - accuracy: 0.6692 - val_loss: 0.3044 - val_accuracy: 0.8645
Epoch 2/2
6/6 [==============================] - 82s 15s/step - loss: 0.2463 - accuracy: 0.8942 - val_loss: 0.2351 - val_accuracy: 0.9030
Fitting the VGG16 model
Epoch 1/2
6/6 [==============================] - 247s 47s/step - loss: 7.7756 - accuracy: 0.7025 - val_loss: 0.9736 - val_accuracy: 0.9145
Epoch 2/2
6/6 [==============================] - 318s 59s/step - loss: 0.7734 - accuracy: 0.9183 - val_loss: 0.9541 - val_accuracy: 0.9055
Fitting the Mobile net model
Epoch 1/2
6/6 [==============================] - 87s 15s/step - loss: 2.2398 - accuracy: 0.4808 - val_loss: 3.2895 - val_accuracy: 0.5010
Epoch 2/2
6/6 [==============================] - 70s 13s/step - loss: 2.7679 - accuracy: 0.5085 - val_loss: 1.5524 - val_accuracy: 0.5415


In [9]:
from sklearn.metrics import classification_report
import numpy as np

y_true = validation_generator.classes

y_pred_new_model = np.argmax(new_model.predict(validation_generator), axis=1)
y_pred_v16 = np.argmax(v16_model.predict(validation_generator), axis=1)
y_pred_mob_net = np.argmax(mobile_net.predict(validation_generator), axis=1)

print('classification report for resnet model')
print(classification_report(y_true, y_pred_new_model))
print('classification report for vgg16 model')
print(classification_report(y_true, y_pred_v16))
print('classification report for mobile net model')
print(classification_report(y_true, y_pred_mob_net))

21/21 [==============================] - 34s 1s/step
classification report for resnet model
              precision    recall  f1-score   support

           0       0.49      0.52      0.50      1011
           1       0.48      0.45      0.47      1012

    accuracy                           0.49      2023
   macro avg       0.49      0.49      0.48      2023
weighted avg       0.49      0.49      0.48      2023

classification report for vgg16 model
              precision    recall  f1-score   support

           0       0.49      0.45      0.47      1011
           1       0.49      0.54      0.52      1012

    accuracy                           0.49      2023
   macro avg       0.49      0.49      0.49      2023
weighted avg       0.49      0.49      0.49      2023

classification report for mobile net model
              precision    recall  f1-score   support

           0       0.50      0.95      0.65      1011
           1       0.50      0.05      0.09      1012

    accur

# Fine Tune models


In [10]:
# with tf.device('/GPU:0'):
    #Unfreeze the base_model layers
new_model.layers[0].trainable = True
# Due to the large size of the data_set, halve the steps per epoch to reduce training time
steps_per_epoch = len(train_generator)//2

new_model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.0001),  # Very low learning rate
                loss=keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=[keras.metrics.BinaryAccuracy()])
new_model.fit(train_generator, epochs=2, steps_per_epoch=steps_per_epoch, validation_data=validation_generator, validation_steps=validation_steps)
   
v16_model.layers[0].trainable = True
 # Due to the large size of the data_set, halve the steps per epoch to reduce training time
steps_per_epoch = len(train_generator)//2

v16_model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.0001),  # Very low learning rate
                loss=keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=[keras.metrics.BinaryAccuracy()])
v16_model.fit(train_generator, epochs=2, steps_per_epoch=steps_per_epoch, validation_data=validation_generator, validation_steps=validation_steps)

mobile_net.layers[0].trainable = True
    # Due to the large size of the data_set, halve the steps per epoch to reduce training time
steps_per_epoch = len(train_generator)//2

mobile_net.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.0001),  # Very low learning rate
                loss=keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=[keras.metrics.BinaryAccuracy()])
mobile_net.fit(train_generator, epochs=2, steps_per_epoch=steps_per_epoch, validation_data=validation_generator, validation_steps=validation_steps)


Epoch 1/2


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\backend.py:5818: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


20/20 [==============================] - 467s 20s/step - loss: 0.1820 - binary_accuracy: 0.9367 - val_loss: 0.2384 - val_binary_accuracy: 0.9465
Epoch 2/2
20/20 [==============================] - 433s 22s/step - loss: 0.0858 - binary_accuracy: 0.9685 - val_loss: 0.1351 - val_binary_accuracy: 0.9645
Epoch 1/2
20/20 [==============================] - 957s 48s/step - loss: 1.1071 - binary_accuracy: 0.5359 - val_loss: 0.6901 - val_binary_accuracy: 0.5320
Epoch 2/2
20/20 [==============================] - 790s 40s/step - loss: 0.7631 - binary_accuracy: 0.5343 - val_loss: 0.7773 - val_binary_accuracy: 0.5010
Epoch 1/2
20/20 [==============================] - 117s 5s/step - loss: 0.3280 - binary_accuracy: 0.8725 - val_loss: 1.3508 - val_binary_accuracy: 0.6645
Epoch 2/2
20/20 [==============================] - 103s 5s/step - loss: 0.1858 - binary_accuracy: 0.9273 - val_loss: 0.7944 - val_binary_accuracy: 0.7840
